# Homework 2 : CUAHSI VU : Snow Hydrology

We'll use the setup from the `homework_2_B_setup.ipynb` notebook for this exercise as well. If you think that you comprised that setup in a previous exercise, just rerun `homework_2_B_setup.ipynb` before continuing.

As before, remember that you want to save your notebooks in a place other than the `examples` directory tree, because files you save there may be deleted and overwritten the next time you start your virtual machine. Save your files somewhere in the `work` directory tree: `work/homework_2` would be a great place for them. So click on "_File --> Save Notebook As..._" and save this notebook as `work/homework_1/homework_2_C_exercise_1.ipynb` before you continue. If things go hopelessly wrong, you can always start over again with the original notebook in your `examples` directory tree.

## C. Homework 2 Exercise 1

### Setup

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt # if you want to do some plotting using matplotlib (optional)
from pysumma.Simulation import Simulation # create Simulation objects
from pysumma.hydroshare import hydroshare # create hydroshare objects to upload to HydroShare
from pysumma.LocalParamInfo import LocalParamInfo # modify local parameters.
import numpy as np # provide some mathematical functions
import xarray as xr # read, write, manipulate and analyze NetCDF files

In [ ]:
resource_id = '05adaf5618ee4dbbbf04195db376cbb4'
case_path = '/home/jovyan/work/{}/{}/data/contents/umpqua'.format(resource_id, resource_id)
filemanager_path = case_path+'/settings/snow_fileManager.txt'
summa_code_path = '/opt/summa'

### Functions

In this section we will pre-define some functions we will use in the rest of the notebook to modify the forcings. In this first exercise we will look at the effects of different methods to estimate incoming longwave radiation on the snow simulations. To do this, we will read the original forcing file (identified by the filename modifier `9697`) into an `xarray Dataset` and we will then calculate new longwave fluxes, which we will write to a new file. We then modify the SUMMA input file that points to the forcing file to be used (the [forcing file list](https://summa.readthedocs.io/en/latest/input_output/SUMMA_input/#list-of-forcing-files-file)). We will show an example below.

#### Update the forcing file list

The following function will update the forcing file list so that SUMMA will read the right forcing file.

In [ ]:
def update_force_file_list(s, new_force_filename):
    '''Update the forcing file list
       s: SUMMA simulation instance
       new: new forcing filename
       Note this only works if there is a single forcing file'''
    fp = s.setting_path.get_value() + s.forcing_list.get_value()
    with open(fp, 'r') as f:
        lines = f.readlines()
    lines[-1] = "'{}'".format(new_force_filename.split('/')[-1])
    with open(fp, 'w') as f:
        f.writelines(lines)

#### Calculate incoming longwave radiation

##### Calculation of vapor pressure from specific humidity

Most methods for estimating incoming longwave radiation from other variables require some combination of air temperature, a measure of humidity (vapor pressure or specific humidity), and perhaps some other variables. First, we will derive the relationship between specific humidity and vapor pressure, since we have specific humidity as part of the SUMMA forcing file.

We begin with definitions of relative humidity and rearranging,

$$ RH = \frac{w}{w_s} = \frac{e}{e_s} \rightarrow e = \frac{e_s \cdot w}{w_s} $$

Given specific humidity as the mass mixing ratio of water vapor we can rearrange, 

$$q = \frac{w}{w+1} \rightarrow w = -\frac{q}{q-1}$$

And finally, using the conversion factor:

$$ w_s \approx 0.622 \frac{e_s}{p} $$

we are able to write the vapor pressure in terms of the specific humidity and air pressure:

$$ e = -\frac{e_s}{w_s}\left(\frac{q}{q-1}\right) \approx -1.607 \cdot p \left(\frac{q}{q-1}\right) $$

With this we can define a number of functions for estimating longwave radiation.

##### Methods for estimating longwave radiation 

We define the following five functions for estimating longwave radiation (the references are provide in the functions): 
* Prata
* TVA
* Satterlund
* Anderson
* Dokia

For each of the functions, the units of the inputs are consistent with the units in the SUMMA forcing files. Note that there are also a few helper functions that are used by one or more of the methods.

In [ ]:
# Stephan-Boltzmann constant (J/s/m^2/K^4)
STEFAN = 5.67e-8


def vapor_pressure(air_pressure, spec_humid):
    """See above for derivation"""
    return -1.607 * air_pressure * spec_humid / (spec_humid - 1)


def longwave_prata(air_temp, vapor_pressure):
    """
    Reference:
        Prata, A.J., 1996. A new long-wave formula for estimating
        downward clear-sky radiation at the surface. Q. J. R. Meteor.
        Soc. 122 (533), 1127–1151, doi:10.1002/qj.49712253306.
    """
    z = 46.5 * (vapor_pressure / air_temp)
    emissivity = 1 - (1 + z) * np.exp(-np.sqrt(1.2 + 3 * z))
    return emissivity * np.power(air_temp, 4) * STEFAN
        
    
def longwave_tva(air_temp, vapor_pressure):
    """
    Reference:
        Tennessee Valley Authority, 1972. Heat and mass transfer between a
        water surface and the atmosphere. Tennessee Valley Authority, Norris,
        TN. Laboratory report no. 14. Water resources research report no. 0-6803.
    """
    emissivity = 0.74 + 0.0049 * vapor_pressure/10
    return emissivity * np.power(air_temp, 4) * STEFAN


def longwave_satterlund(air_temp, vapor_pressure):
    """
    Reference:
        Satterlund, D.R., 1979. An improved equation for estimating long-wave
        radiation from the atmosphere. Water Resour. Res. 15 (6), 1649–1650,
        doi:10.1029/WR015i006p01649.
    """
    vp = vapor_pressure / 10
    emissivity = 1.08 * (1 - np.exp(-np.power(vp, air_temp/2016)))
    return emissivity * np.power(air_temp, 4) * STEFAN


def longwave_anderson(air_temp, vapor_pressure):
    """
    Referencce:
        Anderson, E.R., 1954. Energy budget studies, water loss
        investigations: lake Hefner studies. U.S. Geol. Surv. Prof. Pap. 269,
        71–119 [Available from U.S. Geological Survey, 807 National Center,
        Reston, VA 20192.].
    """
    emissivity = 0.68 + 0.036 * np.power(vapor_pressure/10, 0.5)
    return emissivity * np.power(air_temp, 4) * STEFAN


def cloud_correction(shortwave, lat=43.03, highlimit=0.6, lowlimit=0.35):
    """Converted from cloudfactor_Jessica.m"""
    doy = shortwave.time.dt.dayofyear
    s0 = 1360                     # Solar constant (W/m^2)
    phi = lat * 2 * np.pi / 365   # Convert to radian
    # Declination in radians
    delta = (2 * np.pi / 365) * (23.45 * np.sin(2 * np.pi * (284 + doy) / 365))
    
    # Top of atmosphere radiation
    hs = np.arccos(-np.tan(phi) * np.tan(delta))
    q0 = s0 * (1/np.pi) * (
        hs * np.sin(phi) * np.sin(delta) + np.cos(phi) * np.cos(delta) * np.sin(hs))
    
    # Fraction of recieved radiation
    k = shortwave / q0
    
    # Cloud cover fraction
    cloud_frac = 1 - ((k - lowlimit) / (highlimit - lowlimit))
    cloud_frac = cloud_frac.where(cloud_frac > 0, other=0.0)
    cloud_frac = cloud_frac.where(cloud_frac < 1, other=1.0)
    return cloud_frac
    
    
def longwave_dokia(air_temp, vapor_pressure, shortwave):
    """
    References:
      -Clear sky:
        Dilley, A. C., and D. M. O<92>Brien (1998), Estimating downward clear sky
        long-wave irradiance at the surface from screen temperature and precipitable
        water, Q. J. R. Meteorol. Soc., 124, 1391<96> 1401.
      -Cloudy sky:
        Kimball, B. A., S. B. Idso, and J. K. Aase (1982), A model of thermal                                                                                          
        radiation from partly cloudy and overcast skies, Water Resour. Res., 18,                                                                                       
        931<96> 936.                                   
    """
    vp = vapor_pressure / 1000 # Convert to kPa
    w = 4560 * (vp / air_temp) # Prata (1996) approximation for precipitable water
    
    # Clear sky component of longwave
    lw_clear = 59.38 + 113.7 * np.power(air_temp / 273.16, 6) + 96.96 * np.sqrt(w / 25)
    
    # Cloud cover corrections
    c = cloud_correction(shortwave)
    cloud_temp = air_temp - 11
    winter = np.logical_or(vp.time.dt.month <= 2, vp.time.dt.month == 12)
    summer = np.logical_and(vp.time.dt.month <= 8, vp.time.dt.month >=6)
    cloud_temp[winter] -= 2
    cloud_temp[summer] += 2
    
    # Cloudy sky component of longwave
    eps8z = 0.24 + 2.98e-6 * np.power(vp, 2.0) * np.exp(3000/air_temp)
    tau8 = 1 - eps8z * (1.4 - (0.4 * eps8z))
    f8 = -0.6732 + 0.6240e-2 * cloud_temp - 0.914e-5 * np.power(cloud_temp, 2.0)
    lw_cloud = tau8 * c * f8 * STEFAN * np.power(cloud_temp, 4)   
    return lw_clear + lw_cloud

### Example

Create a SUMMA simulation object.

In [ ]:
S = Simulation(case_data=filemanager_path, summa_code=summa_code_path)

Open the forcing file for this simulation and read the file into an `xarray Dataset` object. Note that the name of the forcing file is stored in the forcing list file. The location of that file is given by:

In [ ]:
forcing_file_list = S.file_dir+'/'+S.forcing_list.filename
forcing_file_list

If you look in that file, you will see that the name of the forcing file is `Storck_AboveCanopyPP_9697.nc`. The directory for that file is given by

In [ ]:
S.input_path.filepath

We can use `xarray` to read the file into an `xarray Dataset`.

In [ ]:
force_file_obs  = S.input_path.filepath + '/' + 'Storck_AboveCanopyPP_9697.nc'
force_obs = xr.open_dataset(force_file_obs)

To examine the contents of the file, you can simply type the name of the `Dataset`

In [ ]:
force_obs

Note that this file contains two hrus (hydrologic response units). In this case, the forcing variables for the two hrus are identical. We can quickly examine this by subtracting the forcing data for the two hrus and taking the mean (just showing some `xarray` features. The difference between the two datasets is the vegetation type. The first hru (hru=0 or hruId=1001) has grass as its vegetation cover, while the second hru (hru=1 or hruId=1002) has evergreen needleleaf forest as its vegetaion cover.

In [ ]:
(force_obs.isel(hru=0)-force_obs.isel(hru=1)).mean()

As you can see, subtracting the two hrus in the `xarra Dataset` will actually subtract all the forcing fields that are in that dataset.

Plotting a time series is also straightforward. For example, a time series of longwave for the first hru can be plotted by

In [ ]:
force_obs.LWRadAtm.isel(hru=0).plot()

Now let's generate a new forcing dataset based on one of the longwave algorithms. Here we'll use Anderson as an example, which takes air temperature and vapor pressure as input (so we first need to calculate the vapor pressure). 

In [ ]:
vp = vapor_pressure(force_obs['airpres'], force_obs['spechum'])
lw_anderson = longwave_anderson(force_obs['airtemp'], vp)

And let's plot the two different longwave time series for the first hru.

In [ ]:
force_obs.LWRadAtm.isel(hru=0).plot(label='Observed')
lw_anderson.isel(hru=0).plot(label='Anderson')
plt.legend()

We'll do a quick run with the default forcings

In [ ]:
ds_obs, outfile_obs = S.execute(run_suffix='lw_obs', run_option='local')

Finally, we'll write our new Anderson forcing to file and we will do simulations with the new longwave time series (the other forcings will remain unchanged).

Writing this to file, we'll use a few steps:
* copy the original forcing dataset to a new one
* replace the longwave values
* write a new forcing file
* replace the forcing file name in the forcing file list with the name of the new file

Once done we can just do the new simulation. Note that in this case we are re-using the existing simulation object `S`. We could use a somewhat different workflow here and create new simulation objects, but for now we recommend this workflow, because it is easy to get the input files mixed up.

In [ ]:
force_anderson = force_obs.copy() # copy the original forcing dataset to a new one

In [ ]:
force_anderson['LWRadAtm'].values = lw_anderson.values # replace the longwave values

In [ ]:
force_file_anderson = force_file_obs.replace('9697', 'anderson')
force_anderson.to_netcdf(force_file_anderson, format='NETCDF3_CLASSIC')

In [ ]:
update_force_file_list(S, force_file_anderson)

In [ ]:
ds_anderson, out_anderson = S.execute(run_suffix='lw_anderson', run_option='local')

**Keep in mind that the file with the forcing file list now points to the Anderson longwave file. If you want to rerun with any other forcing you will need to make sure that the file reported in `forcing_file_list` points to the right input file.**

You can navigate to this file in the file browser to the left.

In [ ]:
forcing_file_list

We can now compare the results from the two simulations.

In [ ]:
plt.figure(figsize=(8,5))
ds_obs.scalarSWE.isel(hru=0).plot(label='Observed longwave - grassland')
ds_anderson.scalarSWE.isel(hru=0).plot(label='Anderson longwave - grassland')
ds_obs.scalarSWE.isel(hru=1).plot(label='Observed longwave - trees')
ds_anderson.scalarSWE.isel(hru=1).plot(label='Anderson longwave - trees')
plt.legend()

### Exercise

Now it is up to you. Use different empirical methods to estimate the incoming longwave irradiance and compare the simulations with each other. Document the impact on the simulated snowpack in a forest clearing and under a forest canopy.

In the next exercise we take this a step further, but for now we just want to ensure that you can create new longwave forcings, use them as input to SUMMA and compare the impacts of these changes on snow pack development.

Finally a few hints and things to keep in mind: It is easy to get confused with the model setup and the simulations. If you make changes to the forcings and the simulations do not change, make sure that your simulation object actually reads the right forcing file.

All the forcing files should end up in the directory : `/home/jovyan/work/05adaf5618ee4dbbbf04195db376cbb4/05adaf5618ee4dbbbf04195db376cbb4/
data/contents/umpqua/forcing/`
You can inspect that directory in the file browser to the left (you can also do that in a terminal if you are familiar with unix).

To make sure that the right file is actually being read, you can inspect the `forcing_file_list`, which is at:
`/home/jovyan/work/05adaf5618ee4dbbbf04195db376cbb4/05adaf5618ee4dbbbf04195db376cbb4/
data/contents/umpqua/settings/snow_zForcingFileList.txt`

Keep in mind that commands in a jupyter notebook can be executed in any order, but that that may affect the results. When things don't make sense, re-install the setup for homework 2 (execute the cells in `homework_2_B_setup.ipynb`) and rerun this notebook. To make sure you start afresh, you should also restart the kernel (so there are no old versions in memory). You can do this by going to `Kernel --> Restart Kernel...` and re-executing the notebook cells or selecting `Kernel --> Restart Kernel and Run All Cells`. Normally you would not redo all the model simulations every time, but since the simulations are short, we'll use that method for now.

## On to the next step

That's it for the first part of the second homework as far as the model simulations go. Prof. Lundquist will talk more about the analysis that you need to do on these simulations. You can do the analysis here in this notebook if python is your analysis software of choice or you can upload your output to HydroShare (in NetCDF or csv format) to do additional analysis.

Save this notebook and close the tab. You can also right-click on the file in the left panel if it has a green dot next to it and select "_Shutdown kernel_" from the popup menu to stop the python session that is executing the commands in this notebook. Then open the next notebook: 
`work/homework_2/homework_2_D_exercise_2.ipynb`.

After you finish this notebook, you can rerun `homework_2_B_setup.ipynb` to restore your settings to those you started with to avoid confusion.